# batch makes all queries to get top 40 products relevant for each query

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
import sys
sys.executable

'/opt/conda/envs/p312/bin/python'

In [3]:
model = 'gemini'
locale = 'us'

In [4]:
q = pd.read_parquet(f'{model}/queries-{locale}.parquet')

In [5]:
p = pd.read_parquet(f'{model}/products-{locale}.parquet')

In [6]:
q.shape, p.shape

((97345, 2), (1215851, 2))

In [7]:
qe = np.stack(q.embeddings.values.copy())
pe = np.stack(p.embeddings.values.copy())
qe.shape, pe.shape

((97345, 768), (1215851, 768))

In [8]:
def get_dotp_closests_idxs(q_embedding,p_embeddings_matrix, topk=40):
    return np.argsort( p_embeddings_matrix.dot(q_embedding) )[-topk:][::-1]


In [9]:
1

1

In [10]:
qnn = Parallel(n_jobs=-1, verbose=5)(delayed(get_dotp_closests_idxs)(qi,pe) for qi in qe)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 322 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 520 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done 754 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1330 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1672 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2050 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 2464 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 2914 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 3400 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 3922 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 4480 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 5074 tasks      | 

In [11]:
q['top40_products'] = [list(p.index[qnni]) for qnni in tqdm(qnn)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 97345/97345 [00:01<00:00, 91154.99it/s]


In [17]:
q

,query,embeddings,top40_products
query_id,,,
0,revent 80 cfm,"[-0.01598229, -0.046302106, -0.013507076, -0.0...","[B074RDGKH4, B008OHPUYM, B004KLWPI0, B0057GS2W..."
1,!awnmower tires without rims,"[-0.030829305, -0.08287808, 0.04729122, 0.0214...","[B084MBY16Z, B0752SYKVX, B077QMNXTS, B076BQCV3..."
2,!qscreen fence without holes,"[-0.028439859, -0.023042167, -0.051055882, 0.0...","[B01N1P9MYW, B01DYR0XAC, B07GXPTGVX, B07BR5HYW..."
5,# 10 self-seal envelopes without window,"[0.04445911, -0.056469392, -0.0016481074, -0.0...","[B07CXXVXLC, B071R9SBXJ, B01N175R8R, B078HHGY2..."
6,# 2 pencils not sharpened,"[0.0021929916, 0.0144573795, 0.00267528, -0.04...","[B07GJQJFG6, B07G2RYY6H, B07JZJLHCF, B07G7F6JZ..."
...,...,...,...
129275,茶叶,"[-0.010790561, 0.037765387, 0.00715581, 0.0184...","[4794217048, B0893CKBJL, B0893T4GDX, B07WSFFH7..."
130378,香奈儿,"[-0.010790561, 0.037765387, 0.00715581, 0.0184...","[4794217048, B0893CKBJL, B0893T4GDX, B07WSFFH7..."
130537,가마솥,"[-0.010790561, 0.037765387, 0.00715581, 0.0184...","[4794217048, B0893CKBJL, B0893T4GDX, B07WSFFH7..."


In [15]:
qr = []
for query_id, qi in tqdm(q.iterrows()):
    for product_id in qi.top40_products:
        qr.append([query_id, product_id])
    

97345it [00:07, 13516.94it/s]


In [16]:
pd.DataFrame(qr, columns = ['query_id', 'product_id']).to_csv(f'{model}/embeddings_dotp_ranking_{locale}.csv', index=False)